In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [43]:
data = pd.read_csv('./data.csv')

In [44]:
data.head()

,seq,label
0,"No, no\r\nI ain't ever trapped out the bando\r...",0.626
1,"The drinks go down and smoke goes up, I feel m...",0.630
2,She don't live on planet Earth no more\r\nShe ...,0.240
3,"Trippin' off that Grigio, mobbin', lights low\...",0.536
4,"I see a midnight panther, so gallant and so br...",0.371


In [45]:
import re
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [46]:
x = []
sentences = list(data["seq"])
for sen in sentences:
    x.append(preprocess_text(sen))

In [47]:
labels = data['label']
y = labels.values

In [48]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x)

x = tokenizer.texts_to_sequences(x)
# x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 500

# X_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [49]:
x = pad_sequences(x, padding='post', maxlen=maxlen)

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [52]:
from sklearn.tree import DecisionTreeClassifier

In [53]:
from sklearn.linear_model import LinearRegression

In [54]:
lr = LinearRegression()

In [70]:
lr.fit(x_train, y_train)

LinearRegression()

In [71]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, lr.predict(x_test))

0.20939078395814806

In [57]:
import xgboost as xgb

In [58]:
boost = xgb.XGBRegressor()

In [59]:
boost.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [60]:
mean_absolute_error(y_test, boost.predict(x_test))

0.20085106100813585

In [61]:
from sklearn.ensemble import GradientBoostingRegressor

In [62]:
gbr = GradientBoostingRegressor()

In [63]:
gbr.fit(x_train, y_train)

GradientBoostingRegressor()

In [66]:
mean_absolute_error(y_test, gbr.predict(x_test))

0.20552753988379868

In [67]:
mean_absolute_error(y_train, gbr.predict(x_train))

0.20243966973888863

In [68]:
mean_absolute_error(y_train, boost.predict(x_train))

0.17233539625933367

In [72]:
mean_absolute_error(y_train, lr.predict(x_train))

0.20670051887526258

In [73]:
import pickle as pkl
with open("xgvalence.pkl", "wb") as file:
    pkl.dump(boost, file)